# Validation Examples

As a result of optimizations, and the low level nature of the Power Grid Model's mathematical core, the core exceptions may not always be clear to the user. Therefore an optional validation mechanism is supplied, which validates data structures and values off-line. It is recommended to always validate your data before constructing a PowerGridModel instance. An alternative approach would be to validate only when an exception is raised, but be aware that not all data errors will raise exceptions: most of them will just yield invalid results without warning.

The basic methods and class definitions are available in the `power_grid_model.validation` module:

```python
# Manual validation
#   validate_input_data() assumes that you won't be using update data in your calculation.
#   validate_batch_data() validates input_data in combination with batch/update data.
validate_input_data(input_data, calculation_type, symmetric) -> list[ValidationError]
validate_batch_data(input_data, update_data, calculation_type, symmetric) -> dict[int, list[ValidationError]]

# Assertions
#   assert_valid_input_data() and assert_valid_batch_data() raise a ValidationException,
#   containing the list/dict of errors, when the data is invalid.
assert_valid_input_data(input_data, calculation_type, symmetric) raises ValidationException
assert_valid_batch_data(input_data, calculation_type, update_data, symmetric) raises ValidationException

# Utilities
#   errors_to_string() converts a set of errors to a human readable (multi-line) string representation
errors_to_string(errors, name, details)
```

Each validation error is an object which can be converted to a compact human-readable message using `str(error)`. It
contains three member variables `component`, `field` and `ids`, which can be used to gather more specific information about the validation error, e.g. which object IDs are involved.

```python
class ValidationError:
    
    # Component(s): e.g. ComponentType.node or [ComponentType.node, ComponentType.line]
    component: ComponentType | list[ComponentType]
    
    # Field(s): e.g. "id" or ["line_from", "line_to"] or [(ComponentType.node, "id"), (ComponentType.line, "id")]
    field: str | list[str] | list[tuple[ComponentType, str]]

    # IDs: e.g. [1, 2, 3] or [(ComponentType.node, 1), (ComponentType.line, 1)]
    ids: list[int] | list[tuple[ComponentType, int]] = []    
    
```

Note: The data types of `input_data` and `update_data` are the same as expected by the power grid model.

In [ ]:
from power_grid_model import ComponentType, DatasetType, PowerGridModel, initialize_array

# A power grid containing several errors

# node
node_error = initialize_array(DatasetType.input, ComponentType.node, 3)
node_error["id"] = [1, 2, 3]
node_error["u_rated"] = [10.5e3]

# line
line_error = initialize_array(DatasetType.input, ComponentType.line, 3)
line_error["id"] = [4, 5, 6]
line_error["from_node"] = [1, 2, 3]
line_error["to_node"] = [2, 3, 4]
line_error["from_status"] = [True]
line_error["to_status"] = [True]
line_error["r1"] = [0.25]
line_error["x1"] = [0.2]
line_error["c1"] = [10e-6]
line_error["tan1"] = [0.0]

# Power Sensor
sensor_error = initialize_array(DatasetType.input, ComponentType.sym_power_sensor, 2)
sensor_error["id"] = [6, 7]
sensor_error["measured_object"] = [3, 4]
sensor_error["measured_terminal_type"] = [0, 2]
sensor_error["p_measured"] = [0]
sensor_error["q_measured"] = [0]
sensor_error["power_sigma"] = [0]

error_data = {
    ComponentType.node: node_error,
    ComponentType.line: line_error,
    ComponentType.sym_power_sensor: sensor_error,
}

In [ ]:
# Without validation
model = PowerGridModel(error_data)
output_data = model.calculate_state_estimation(symmetric=True)

In [ ]:
from power_grid_model.validation import assert_valid_input_data

# Assert valid data
assert_valid_input_data(error_data, symmetric=True)
model = PowerGridModel(error_data)
output_data = model.calculate_state_estimation(symmetric=True)

In [ ]:
from power_grid_model.validation import ValidationException

# Assert valid data and display component ids
try:
    assert_valid_input_data(error_data, symmetric=True)
    model = PowerGridModel(error_data)
    output_data = model.calculate_state_estimation(symmetric=True)
except ValidationException as ex:
    for error in ex.errors:
        print(type(error).__name__, error.component, ":", error.ids)

In [ ]:
from power_grid_model.validation import errors_to_string, validate_input_data

# Validation only as exception handling
try:
    model = PowerGridModel(error_data)
    output_data = model.calculate_state_estimation(symmetric=True)
except RuntimeError as _ex:
    errors = validate_input_data(error_data, symmetric=True)
    print(errors_to_string(errors))

In [ ]:
# Manual checking and display detailed information about the invalid data
errors = validate_input_data(error_data, symmetric=True)
print(errors_to_string(errors, details=True))